## 2.1 超参选择 - 交叉验证
交叉验证（Cross-Validation）是一种用于评估机器学习模型性能的技术，特别是在超参数选择过程中。它通过将数据集划分为多个子集，反复训练和验证模型，以获得更可靠的性能估计。
常见的交叉验证方法包括K折交叉验证（K-Fold Cross-Validation）和留一法交叉验证（Leave-One-Out Cross-Validation）。
在K折交叉验证中，数据集被划分为K个子集（折），每次使用K-1个子集进行训练，剩下的一个子集用于验证。这个过程重复K次，每个子集都被用作验证集一次。最终的模型性能是K次验证结果的平均值。

In [1]:
import numpy as np
from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

#### 1. 划分数据集

In [2]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### 2. 创建K折交叉验证对象

In [3]:
# 创建cv对象
from sklearn.model_selection import KFold
cv = KFold(n_splits=5, shuffle=True, random_state=42) # 5折交叉验证, 将训练集划分为5个子集，shuffle=True表示在划分前打乱数据顺序

# 候选的k值
k_list = [3, 5, 7, 9] # 通常为奇数，避免平局，并且不超过训练集样本数的一半

#### 3. 对每一个K，手动实现多次（不使用网格搜索 GridSearchCV）K折交叉验证

In [6]:
# 创建KNN模型，使用Pipline创建，因为训练集的标准化和KNN模型训练需要在每个折内进行，使用Pipline可以避免数据泄漏
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score
import numpy as np

scores_mean = []
scores_std = []

# 手动尝试每个k值
for k in k_list:
    # 创建KNN模型
    pipline = Pipeline([
        ("scaler", StandardScaler()),  # 标准化
        ("knn", KNeighborsClassifier(n_neighbors=k, metric="minkowski", p=2))  # KNN模型, 欧氏距离
    ])
    # 在每个折上进行训练和验证，并且储存sores_mean和scores_std
    scores = cross_val_score(pipline, X_train, y_train, cv=cv, scoring="accuracy")
    scores_mean.append(scores.mean())
    scores_std.append(scores.std())
    print("k =", k, "-> scores:", scores, "-> mean:", scores.mean(), "-> std:", scores.std())

# 找到最佳k值
best_k = k_list[np.argmax(scores_mean)]
print("Best k:", best_k)

k = 3 -> scores: [0.91666667 0.95833333 0.91666667 0.83333333 1.        ] -> mean: 0.925 -> std: 0.055277079839256664
k = 5 -> scores: [0.91666667 0.95833333 0.91666667 0.91666667 1.        ] -> mean: 0.9416666666666667 -> std: 0.033333333333333354
k = 7 -> scores: [0.91666667 1.         0.91666667 0.91666667 1.        ] -> mean: 0.95 -> std: 0.04082482904638632
k = 9 -> scores: [0.91666667 1.         0.875      0.91666667 1.        ] -> mean: 0.9416666666666667 -> std: 0.05
Best k: 7


#### 4. 使用最佳k值训练最终模型并评估性能

In [7]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
# 使用最佳k值训练最终模型
best_model = Pipeline([
    ("scaler", StandardScaler()),
    ("knn", KNeighborsClassifier(n_neighbors=best_k, metric="minkowski", p=2))
])
best_model.fit(X_train, y_train)
# 在测试集上评估性能
y_pred = best_model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Test set accuracy with best k (k={}): {:.2f}%".format(best_k, accuracy * 100))
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Test set accuracy with best k (k=7): 100.00%
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        10
           1       1.00      1.00      1.00         9
           2       1.00      1.00      1.00        11

    accuracy                           1.00        30
   macro avg       1.00      1.00      1.00        30
weighted avg       1.00      1.00      1.00        30

Confusion Matrix:
 [[10  0  0]
 [ 0  9  0]
 [ 0  0 11]]
